# Data Augmentation Lab

Welcome to the data augmentation lab! By the end of this lab you will have

- Used data augmentation to increase the effective size a training set resulting in increased performance on MNIST

Let's get started!

# Load MNIST

The following code loads MNIST images into memory and normalizes them.

In [ ]:
from IPython.display import display
from keras.datasets import mnist

[X_mnist, y_mnist], [X_test, y_test] = mnist.load_data()
X_mnist, X_test = X_mnist/255., X_test/255.
    
X_mnist.shape

# Create Training Set

The following code creates a training set consisting of one image digit per class.

In [ ]:
from IPython.display import display
from keras.preprocessing.image import array_to_img
import numpy as np

digit_idxs = [np.argwhere(y_mnist == d).flatten()[0] for d in range(10)]
X_train = X_mnist[digit_idxs]

for x in X_train:
    img = array_to_img(np.expand_dims(x, axis=0), data_format='channels_first')
    display(img)

X_train.shape

## Define the CNN Architecture

The following code defines and compiles a CNN architecture. You are **not** to modify it!

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[28, 28, 1]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

## Prepare Data for TensorFlow

The following code converts the training and test images into TensorFlow dimension ordering. Additionally we only take the first 1000 test images to form our test set.

In [ ]:
from keras.utils.np_utils import to_categorical

nb_test = 1000

X_tf_train = np.expand_dims(X_train, axis=-1)
X_tf_test = np.expand_dims(X_test, axis=-1)[:nb_test]

y_train, y_test = np.arange(10), y_test[:nb_test]
Y_train, Y_test = to_categorical(y_train), to_categorical(y_test)

X_tf_train.shape, X_tf_test.shape

# Fit the Model

The following code fits the model defined above on the training set and validates over the test set. A callback is used which displays training information after every epoch.

In [ ]:
import IPython
import keras
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

class TrainingVisualizerCallback(keras.callbacks.History):
    def on_epoch_end(self, epoch, logs={}):
        super().on_epoch_end(epoch, logs)
        IPython.display.clear_output(wait=True)
        pd.DataFrame(self.history).plot()
        plt.show()

model.fit(X_tf_train, Y_train, epochs=25, validation_data=(X_tf_test, Y_test), callbacks=[TrainingVisualizerCallback()])

## Task

- Play around with `keras.preprocessing.image.ImageDataGenerator` to generate some images which look like realistic MNIST images

## Hints

- Learn what types of augmentations are available by reading the [`keras.preprocessing.image.ImageDataGenerator`](https://keras.io/preprocessing/image/) documentation

In [ ]:
from IPython.display import display
import keras

datagen = keras.preprocessing.image.ImageDataGenerator()

image = datagen.flow(X_tf_train, y_train, batch_size=1)
for _ in range(10):
    X, y = next(image)
    img = array_to_img(X[0])
    display(img)

## Task

- Use data augmentation to improve over the baseline performance of using just a single training example from each class

## Hints

- Use [`keras.preprocessing.image.ImageDataGenerator`](https://keras.io/preprocessing/image/) for performing data augmentation to increase the effective size of your training set

## Requirements

- Use the hyperparameters above and do **not** modify any of them
- Execute the code cell above which redefines and recompiles the model to reset the weights!

## Concessions

- You may use `X_tf_test` for validation `(`e.g. `model.fit(..., validation_data=(X_tf_test, Y_test))` for simplicity and report this number

# Question

- Did you find any augmentations which resulted in an increase in prediction accuracy? If so, which augmentation(s)? Be sure to include evidence data to back up your claim.